In [1]:
from proximity.network import Network
from itertools import combinations
import pandas as pd
import networkx as nx
import graph_tool.all as gt
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [8]:
#read ppi network
G = nx.from_pandas_edgelist(pd.read_csv('data/Gene_Gene.csv'), 'source', 'target')

In [9]:
#read chemical-target associations
_df = pd.read_csv('data/chemical_target_experiment.csv')
temp = _df.groupby('chemName')['GeneId'].apply(set).to_dict()

In [10]:
net = Network(G)

### separation

In [35]:
chem_colors = {
    'Ascorbic acid': '#008480',
    'Corilagin': '#eba628',
    'Ellagic acid': '#eba628',
    'Gallic acid': '#7644b5',
    'Kaempferol': '#007f36',
    'Pedunculagin': '#eba628',
    'Pyrogallol': '#77c600',
    'Quercetin': '#007f36',
    'Citric acid': '#0e679f'
}

In [50]:
#compute separation
sep_df = []
for a,b in combinations(temp.keys(), 2): sep_df.append([a, b, net.get_separation(temp[a], temp[b])])
sep_df = pd.DataFrame(sep_df, columns=['A', 'B', 'sep'])
sep_df = sep_df[sep_df.sep.notnull()]
sep_df.head(1)

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,A,B,sep
0,Ascorbic acid,Citric acid,0.035632


In [51]:
#complementary exposure
sep_df.sep.apply(lambda x: True if x>0 else False).value_counts()

True     13
False     8
Name: sep, dtype: int64

In [52]:
sep_df = sep_df[sep_df.sep >= 0]

In [54]:
g = gt.Graph(directed=False)
ew = g.new_ep('float')
ec = g.new_ep('string')
#ids = g.add_edge_list(sep_df.drop('sep',axis=1).values, hashed=True, eprops=[ew, ec])
ids = g.add_edge_list(sep_df.values, hashed=True, eprops=[ew, ec])
for e in g.edges(): ew[e] = ew[e] * 30 #* -1

In [55]:
vt = g.new_vp('string')
vc = g.new_vp('string')
for v in g.iter_vertices(): 
    vt[v] = ids[v]
    vc[v] = chem_colors[ids[v]]

In [62]:
pos = gt.sfdp_layout(g)

In [64]:
gt.graph_draw(g, pos=pos, vertex_text=vt, vertex_font_size=10, edge_pen_width=ew, vertex_pen_width=4, 
              vertex_fill_color='#9f9d9d', edge_color='#00854f', vertex_color=vc)#, output='data/out.svg')  #00854f, 840000

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f71dfd16210, at 0x7f72161d3fd0>